# Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

from transformers import pipeline
from concurrent.futures import ThreadPoolExecutor
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Database
from pymongo import MongoClient
from pymongo.errors import BulkWriteError

import requests
from bs4 import BeautifulSoup
import os
import re

# Saving to Database

In [2]:
def save_to_database(collection_name, df):
    """
    Save a DataFrame to a MongoDB database.
    collection_name: The name of the collection to save the data to.
    df: The DataFrame to save to the database.
    """
    # Connect to MongoDB
    client = MongoClient('mongodb://localhost:27017/')
    # Create a database
    db = client['movies']
    # Create a collection
    collection = db[collection_name]
    # Convert the DataFrame to a dictionary
    data_dict = df.to_dict("records")
    # Insert the data into the database
    try:
        collection.insert_many(data_dict, ordered=False)
    except BulkWriteError as e:
        print("There were some duplicates that were not inserted.")

    # Close the connection
    client.close()

# Rotten Tomatoes Critics Reviews

## Reading Rotten Tomatoes Dataset

In [64]:
rotten_tomatoes_movies = pd.read_csv('./datasets/critic_reviews/rotten_tomatoes_movies.csv')
rotten_tomatoes_critic_reviews = pd.read_csv('./datasets/critic_reviews/rotten_tomatoes_critic_reviews.csv')

## Saving to Database 

In [65]:
# removing unnecessary columns
rotten_tomatoes_movies = rotten_tomatoes_movies.drop(
    columns=['movie_info', 'critics_consensus', 'content_rating', 'directors', 'authors',
       'streaming_release_date', 'runtime', 'production_company', 'tomatometer_status',
       'tomatometer_rating', 'tomatometer_count', 'audience_status', 'audience_rating',
       'audience_count', 'tomatometer_top_critics_count',
       'tomatometer_fresh_critics_count', 'tomatometer_rotten_critics_count'])
rotten_tomatoes_movies.to_csv('./datasets/intermediate_results/rotten_tomatoes_movies.csv', index=False)
# saving to database
save_to_database('rotten_tomatoes_movies', rotten_tomatoes_movies)

# removing unnecessary columns
rotten_tomatoes_critic_reviews = rotten_tomatoes_critic_reviews.drop(
    columns=['critic_name', 'top_critic', 'publisher_name',
       'review_type', 'review_score', 'review_date'])
rotten_tomatoes_critic_reviews.to_csv('./datasets/intermediate_results/rotten_tomatoes_critic_reviews.csv', index=False)
# saving to database
save_to_database('rotten_tomatoes_critic_reviews', rotten_tomatoes_critic_reviews)

## Reading data from Database

In [66]:
client = MongoClient('mongodb://127.0.0.1:27017')
db = client["movies"]
rotten_tomatoes_movies_collection = db['rotten_tomatoes_movies']
rotten_tomatoes_critic_reviews_collection = db['rotten_tomatoes_critic_reviews']

# Fetch movies
movies_cursor = rotten_tomatoes_movies_collection.find({}, no_cursor_timeout=True).batch_size(50)
reviews_cursor = rotten_tomatoes_critic_reviews_collection.find({}, no_cursor_timeout=True).batch_size(50)

# read movies from database into a dataframe
rotten_tomatoes_movies_df = pd.DataFrame(list(movies_cursor))
rotten_tomatoes_reviews_df = pd.DataFrame(list(reviews_cursor))

# close the cursor
movies_cursor.close()
reviews_cursor.close()

rotten_tomatoes_movies_df.drop(columns=['_id'], inplace=True)
rotten_tomatoes_reviews_df.drop(columns=['_id'], inplace=True)


/Users/gaurav/anaconda3/envs/prog_for_ai/lib/python3.9/site-packages/pymongo/collection.py:1697: UserWarning:

use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout



## Data Preprocessing

In [67]:
def rotten_tomatoes_movies_preprocessing(input_df):
    """
    Preprocess the rotten_tomatoes_movies dataset
    1. Drop columns that are not needed
    2. Drop rows with missing values
    3. Drop duplicates
    4. Convert original_release_date to datetime
    5. Sort by original_release_date
    6. Reset index
    7. Return the preprocessed dataframe
    """
    df = input_df.copy()
    df = df.dropna()
    df = df.drop_duplicates(subset=['movie_title'])
    df['original_release_date'] = pd.to_datetime(df['original_release_date'])
    df = df.sort_values(by='original_release_date')
    df = df.reset_index(drop=True)
    return df

def rotten_tomatoes_critic_reviews_preprocessing(input_df):
    """
    Preprocess the rotten_tomatoes_critic_reviews dataset
    1. Drop columns that are not needed
    2. Drop rows with missing values
    3. Convert review_date to datetime
    4. Sort by review_date
    5. Reset index
    6. Return the preprocessed dataframe
    """
    df = input_df.copy()
    df = df.dropna()
    df = df.drop_duplicates()
    df = df.reset_index(drop=True)
    return df

In [68]:
rotten_tomatoes_movies_df = rotten_tomatoes_movies_preprocessing(rotten_tomatoes_movies_df)
rotten_tomatoes_reviews_df = rotten_tomatoes_critic_reviews_preprocessing(rotten_tomatoes_reviews_df)

## Merging Movies and reviews dataframes

In [69]:
rotten_tomatoes_critic_reviews_df = pd.merge(rotten_tomatoes_movies_df, rotten_tomatoes_reviews_df, on='rotten_tomatoes_link', how='inner').drop(columns=['rotten_tomatoes_link'])

In [70]:
rotten_tomatoes_critic_reviews_df.rename(columns={'review_content': 'critic_review'}, inplace=True)

## Select movie reviews where movie release year is 2010 or after

In [71]:
rotten_tomatoes_critic_reviews_df = rotten_tomatoes_critic_reviews_df[rotten_tomatoes_critic_reviews_df['original_release_date'].dt.year >= 2010].reset_index(drop=True)

## Sentiment Analysis using "distilbert-base-uncased" model

In [ ]:
"""
Apply sentiment analysis
"""

# Load the sentiment analysis pipeline
classifier = pipeline('sentiment-analysis', model='sarahai/movie-sentiment-analysis')

def apply_sentiment_analysis(review):
    return classifier(review)[0]

# Apply sentiment analysis using ThreadPoolExecutor for parallel processing
with ThreadPoolExecutor() as executor:
    rotten_tomatoes_critic_reviews_df['critic_sentiment'] = list(
        executor.map(apply_sentiment_analysis, rotten_tomatoes_critic_reviews_df
['critic_review']))


## Converting Sentiment Analysis Results

In [ ]:
"""
Visualize the sentiment analysis
Convert the sentiment to 1 if positive, 0 if negative
"""

condition = rotten_tomatoes_critic_reviews_df['critic_sentiment'].apply(lambda x: eval(x)['label'] == 'LABEL_1')
score = rotten_tomatoes_critic_reviews_df['critic_sentiment'].apply(lambda x: eval(x)['score'])

rotten_tomatoes_critic_reviews_df['critic_sentiment'] = np.where(condition, 1, 0)
rotten_tomatoes_critic_reviews_df['critic_sentiment_score'] = score


In [72]:
# To be used if using csv instaed of database

df = pd.read_csv('./rotten_tomatoes_sentiment.csv')
df.rename(columns={'review_content': 'critic_review', 'sentiment': 'critic_sentiment', 'score': 'critic_sentiment_score'}, inplace=True)
colums_to_select = ['critic_sentiment', 'critic_sentiment_score']
rotten_tomatoes_critic_reviews_df = pd.concat([rotten_tomatoes_critic_reviews_df, df[colums_to_select]], axis=1)
rotten_tomatoes_critic_reviews_df.dropna(inplace=True)
rotten_tomatoes_critic_reviews_df = rotten_tomatoes_critic_reviews_df[rotten_tomatoes_critic_reviews_df['original_release_date'].dt.year >= 2010].reset_index(drop=True)

## Saving to Database

In [12]:
rotten_tomatoes_critic_reviews_df.to_csv('./datasets/intermediate_results/rotten_tomatoes_critic_reviews_with_sentiments.csv', index=False)
save_to_database('rotten_tomatoes_critic_reviews_with_sentiments', rotten_tomatoes_critic_reviews_df)

## Reading Data from Database

In [62]:
client = MongoClient('mongodb://127.0.0.1:27017')
db = client["movies"]

rotten_tomatoes_critic_reviews_with_sentiments_collection = db['rotten_tomatoes_critic_reviews_with_sentiments']

# Fetch movies
rotten_tomatoes_critic_reviews_with_sentiments_cursor = rotten_tomatoes_critic_reviews_with_sentiments_collection.find({}, no_cursor_timeout=True).batch_size(50)

# read movies from database into a dataframe
rotten_tomatoes_critic_reviews_df = pd.DataFrame(list(rotten_tomatoes_critic_reviews_with_sentiments_cursor))


# close the cursor
rotten_tomatoes_critic_reviews_with_sentiments_cursor.close()

rotten_tomatoes_critic_reviews_df.drop(columns=['_id'], inplace=True)

/Users/gaurav/anaconda3/envs/prog_for_ai/lib/python3.9/site-packages/pymongo/collection.py:1697: UserWarning:

use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout



In [73]:
rotten_tomatoes_critic_reviews_df

,movie_title,genres,actors,original_release_date,critic_review,critic_sentiment,critic_sentiment_score
0,"My Son, My Son, What Have Ye Done","Drama, Horror, Mystery & Suspense","Michael Shannon, Chloë Sevigny, Willem Dafoe, ...",2010-01-01,An unusually unwatchable effort from filmmaker...,1,0.746352
1,"My Son, My Son, What Have Ye Done","Drama, Horror, Mystery & Suspense","Michael Shannon, Chloë Sevigny, Willem Dafoe, ...",2010-01-01,"Both baffling and mystifying, the film details...",1,0.521713
2,"My Son, My Son, What Have Ye Done","Drama, Horror, Mystery & Suspense","Michael Shannon, Chloë Sevigny, Willem Dafoe, ...",2010-01-01,"My Son, My Son, What Have Ye Done is more cont...",1,0.948853
3,"My Son, My Son, What Have Ye Done","Drama, Horror, Mystery & Suspense","Michael Shannon, Chloë Sevigny, Willem Dafoe, ...",2010-01-01,Herzog has gone beyond Good and Evil to reinve...,0,0.837280
4,"My Son, My Son, What Have Ye Done","Drama, Horror, Mystery & Suspense","Michael Shannon, Chloë Sevigny, Willem Dafoe, ...",2010-01-01,"It's likely that only Herzog would dare to, an...",1,0.968642
...,...,...,...,...,...,...,...
458432,Billy the Kid,"Documentary, Special Interest",Billy P.,2020-09-30,The rough and ready filmmaking technique makes...,0,0.964485
458433,Billy the Kid,"Documentary, Special Interest",Billy P.,2020-09-30,"Billy, who suffers from Aspergers Syndrome, is...",0,0.860325
458434,Billy the Kid,"Documentary, Special Interest",Billy P.,2020-09-30,Billy the Kid lets you see the character first...,0,0.979273
458435,Billy the Kid,"Documentary, Special Interest",Billy P.,2020-09-30,The first 15 minutes of the film are exemplary...,0,0.942884


## Visualizations

In [74]:
rotten_tomatoes_critic_reviews_df = rotten_tomatoes_critic_reviews_df.assign(
    genres=rotten_tomatoes_critic_reviews_df['genres'].apply(lambda x: [genre for genre in x.replace(' &', ',').split(', ')]),
    actors=rotten_tomatoes_critic_reviews_df['actors'].apply(lambda x: x.split(', ')),
    original_release_date = pd.to_datetime(rotten_tomatoes_critic_reviews_df['original_release_date'])
)

In [33]:
# Flatten the 'genres' column
flat_genre_df = rotten_tomatoes_critic_reviews_df.explode('genres')

# Calculate total number of positive and negative reviews for each genre
genre_reviews_counts = flat_genre_df.groupby(['genres', 'critic_sentiment'])['movie_title'].count().reset_index(name='num_reviews')

# Pivot the table to have positive and negative reviews as columns
genre_reviews_pivot = genre_reviews_counts.pivot(index='genres', columns='critic_sentiment', values='num_reviews').fillna(0)

# Calculate percentage of positive and negative reviews for each genre
genre_reviews_pivot['total_reviews'] = genre_reviews_pivot.sum(axis=1)
genre_reviews_pivot['positive_percentage'] = (genre_reviews_pivot[1] / genre_reviews_pivot['total_reviews']) * 100
genre_reviews_pivot['negative_percentage'] = (genre_reviews_pivot[0] / genre_reviews_pivot['total_reviews']) * 100

# Create stacked bar chart
fig = go.Figure()
fig.add_trace(go.Bar(x=genre_reviews_pivot.index, y=genre_reviews_pivot['positive_percentage'], name='Positive', marker_color='green'))
fig.add_trace(go.Bar(x=genre_reviews_pivot.index, y=genre_reviews_pivot['negative_percentage'], name='Negative', marker_color='red'))

# Update layout
fig.update_layout(barmode='stack', title='Percentage of Positive and Negative Reviews by Genre', xaxis_title='Genre', yaxis_title='Percentage')

# Show plot
fig.show()

In [34]:
# Explode the 'genres' column to create separate rows for each genre
exploded_df = rotten_tomatoes_critic_reviews_df.explode('genres')

# Extract year from the original_release_date
exploded_df['year'] = exploded_df['original_release_date'].dt.year

# Group data by year and genre, and calculate sentiment distribution
grouped_data = exploded_df.groupby(['year', 'genres', 'critic_sentiment']).size().unstack(fill_value=0)

# Plot histograms for each genre over the years
for genre in exploded_df['genres'].unique():
    genre_data = grouped_data.loc[grouped_data.index.get_level_values('genres') == genre]

    fig = go.Figure()
    for sentiment in [0, 1]:  # Negative and positive sentiments
        values = genre_data[sentiment].tolist()
        fig.add_trace(go.Bar(x=genre_data.index.get_level_values('year'), y=values, name='Negative' if sentiment == 0 else 'Positive'))

    fig.update_layout(title=f'Sentiment Distribution for Genre: {genre} Over the Years', barmode='stack')
    fig.update_xaxes(title='Year')
    fig.show()

In [37]:
# Explode the 'genres' column to create separate rows for each genre
exploded_df = rotten_tomatoes_critic_reviews_df.explode('genres')

# Extract year from the original_release_date
exploded_df['year'] = exploded_df['original_release_date'].dt.year

# Group data by year and genre, and calculate sentiment distribution
grouped_data = exploded_df.groupby(['year', 'genres', 'critic_sentiment']).size().unstack(fill_value=0)

# Calculate overall, positive, and negative sentiment counts for each genre for each year
genre_sentiment_counts = {}
for genre in exploded_df['genres'].unique():
    genre_sentiment_counts[genre] = {}
    for year in exploded_df['year'].unique():
        try:
            genre_data = grouped_data.loc[(year, genre), :]
            overall_count = genre_data.sum()
            positive_count = genre_data[1]
            negative_count = genre_data[0]
        except KeyError:
            # If the combination doesn't exist, set counts to zero
            overall_count = 0
            positive_count = 0
            negative_count = 0
        genre_sentiment_counts[genre][year] = {'overall': overall_count, 'positive': positive_count, 'negative': negative_count}

# Convert the dictionary to a DataFrame for easier visualization
genre_sentiment_counts_df = pd.DataFrame.from_dict({(i, j): genre_sentiment_counts[i][j] 
                                                     for i in genre_sentiment_counts.keys() 
                                                     for j in genre_sentiment_counts[i].keys()},
                                                   orient='index')

# Stacked Bar Chart
fig = go.Figure()
for genre in genre_sentiment_counts_df.index.levels[0]:
    genre_data = genre_sentiment_counts_df.loc[genre]
    fig.add_trace(go.Bar(x=genre_data.index, y=genre_data['positive'], name=f'{genre} - Positive'))
    fig.add_trace(go.Bar(x=genre_data.index, y=genre_data['negative'], name=f'{genre} - Negative'))

fig.update_layout(barmode='stack', title='Overall Sentiment Counts for Each Genre Over the Years',
                  xaxis_title='Year', yaxis_title='Sentiment Count')
fig.show()

# Line Plot
fig = go.Figure()
for genre in genre_sentiment_counts_df.index.levels[0]:
    genre_data = genre_sentiment_counts_df.loc[genre]
    fig.add_trace(go.Scatter(x=genre_data.index, y=genre_data['positive'], mode='lines', name=f'{genre} - Positive'))
    fig.add_trace(go.Scatter(x=genre_data.index, y=genre_data['negative'], mode='lines', name=f'{genre} - Negative'))

fig.update_layout(title='Trend of Sentiment Counts for Each Genre Over the Years',
                  xaxis_title='Year', yaxis_title='Sentiment Count')
fig.show()


# Area Chart
fig = go.Figure()
for genre in genre_sentiment_counts_df.index.levels[0]:
    genre_data = genre_sentiment_counts_df.loc[genre]
    fig.add_trace(go.Scatter(x=genre_data.index, y=genre_data['positive'], fill='tozeroy', mode='none',
                             name=f'{genre} - Positive'))
    fig.add_trace(go.Scatter(x=genre_data.index, y=genre_data['negative'], fill='tozeroy', mode='none',
                             name=f'{genre} - Negative'))

fig.update_layout(title='Trend of Sentiment Counts for Each Genre Over the Years',
                  xaxis_title='Year', yaxis_title='Sentiment Count', hovermode='x')
fig.show()

# Grouped Bar Chart
fig = go.Figure()

for genre in genre_sentiment_counts_df.index.levels[0]:
    genre_data = genre_sentiment_counts_df.loc[genre]
    fig.add_trace(go.Bar(x=genre_data.index, y=genre_data['positive'], 
                         name=f'{genre} - Positive', offset=-0.25))
    fig.add_trace(go.Bar(x=genre_data.index, y=genre_data['negative'], 
                         name=f'{genre} - Negative', offset=0.25))

fig.update_layout(barmode='group', title='Sentiment Counts for Each Genre Within Each Year',
                  xaxis_title='Year', yaxis_title='Sentiment Count')
fig.show()

# IMDB + TMDB User Reviews

## Load Kaggle IMDB Reviews Data to Mongodb and scrape the review url data

In [21]:
def scrape_reviews(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        reviews_text = soup.get_text()
        review_splits = reviews_text.split('Permalink')
        
        reviews = []
        for review in review_splits:
            lines = review.strip().split('\n')
            if len(lines) > 2:  
                rating = lines[0].strip()
                user_date = lines[1].strip()
                content = ' '.join(lines[2:]).strip()
                reviews.append({
                    'rating': rating,
                    'user_date': user_date,
                    'content': content
                })
        
        return reviews
    except Exception as e:
        print(f"Failed to scrape {url} with error {e}")
        return []

In [ ]:
def insert_movies(directory_path):
    client = MongoClient('localhost', 27017)
    db = client['movies']
    collection = db['imdb_tmdb_reviews']

    # Loop through all CSV files in the directory
    for filename in os.listdir(directory_path):
        if filename.endswith(".csv"):
            csv_path = os.path.join(directory_path, filename)
            df = pd.read_csv(csv_path)
            
            for _, row in df.iterrows():
                # Check if the movie already exists in the database
                if collection.count_documents({'name': row['name']}) == 0:
                    print(f"Processing {row['name']}...")
                    reviews = scrape_reviews(row['review_url'])
                    movie_data = {
                        'name': row['name'],
                        'year': row['year'],
                        'movie_rated': row['movie_rated'],
                        'run_length': row['run_length'],
                        'genres': row['genres'].split(', '), 
                        'release_date': row['release_date'],
                        'rating': row['rating'],
                        'num_raters': row['num_raters'],
                        'num_reviews': row['num_reviews'],
                        'reviews': reviews
                    }
                    collection.insert_one(movie_data)
                    print(f"Inserted {row['name']} into database.")
                else:
                    print(f"Skipping {row['name']} as it is already in the database.")


insert_movies("./datasets/user_reviews")

## Get data from database, perform sentiment analysis and update it in database

In [ ]:
classifier = pipeline('sentiment-analysis', model='sarahai/movie-sentiment-analysis')

client = MongoClient('localhost', 27017)
db = client['movies']
collection = db['imdb_reviews']
movies = collection.find({}, no_cursor_timeout=True).batch_size(50)

def clean_review(text):
    text = re.sub(r'\w+\d+ \w+ \d+', '', text)
    text = re.sub(r'\d{1,3}(,\d{3})* out of \d{1,3}(,\d{3})* found this helpful\.?\s*Was this review helpful\?\s*Sign in to vote\.', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

try:
    for movie in movies:
        print(movie['name'])
        if 'reviews' in movie:
            for review in movie['reviews']:
                cleaned_review = clean_review(review['content'])  
                result = classifier(cleaned_review[:512]) 
                if result[0]['label'] == 'LABEL_1':
                    sentiment = "positive"
                else:
                    sentiment = "negative"
                review['sentiment'] = sentiment
                collection.update_one({'_id': movie['_id']}, {'$set': {'reviews': movie['reviews']}})
finally:
    movies.close()

print("Sentiments added to all reviews.")

## Load The Rotten Tomatoes Data in MongoDB To keep user reviews data and movies data similar

In [ ]:
df = pd.read_csv('./datasets/critic_reviews/rotten_tomatoes_movies.csv')

columns_to_keep = ['movie_title', 'genres', 'directors', 'actors', 'original_release_date', 'production_company']
df = df[columns_to_keep]

df = df.drop_duplicates()

client = MongoClient('mongodb://localhost:27017/') 
db = client['movies']  
collection = db['tmdb_reviews'] 
data_dict = df.to_dict("records")

try:
    collection.insert_many(data_dict, ordered=False)
except BulkWriteError as e:
    print("There were some duplicates that were not inserted.")

client.close()

## Fetch TMDB User Reviews for the Rotten Toamtoes Movies

In [ ]:
client = MongoClient('mongodb://127.0.0.1:27017')
db = client["movies"]
collection = db['tmdb_reviews']
from pymongo import MongoClient
import requests

# Constants
TMDB_API_KEY = '6748a0069777d79ba648978d81157873'
TMDB_API_URL = 'https://api.themoviedb.org/3'
TMDB_SEARCH_URL = f"{TMDB_API_URL}/search/movie"
TMDB_REVIEWS_URL_TEMPLATE = f"{TMDB_API_URL}/movie/{{}}/reviews"


# Fetch movies
movies_cursor = collection.find({}, no_cursor_timeout=True).batch_size(50)

# Function to search movie on TMDB and get movie id
def search_movie_on_tmdb(movie_title):
    params = {
        'api_key': TMDB_API_KEY,
        'query': movie_title
    }
    response = requests.get(TMDB_SEARCH_URL, params=params)
    data = response.json()
    results = data.get('results')
    if results:
        # Return the ID of the first search result
        return results[0]['id']
    return None

# Function to get reviews from TMDB
def get_reviews_from_tmdb(movie_id):
    url = TMDB_REVIEWS_URL_TEMPLATE.format(movie_id)
    params = {'api_key': TMDB_API_KEY}
    response = requests.get(url, params=params)
    return response.json().get('results', [])

# Main loop to process each movie
for movie in movies_cursor:
    movie_title = movie.get('movie_title')
    if movie_title:
        movie_id = search_movie_on_tmdb(movie_title)
        if movie_id:
            reviews = get_reviews_from_tmdb(movie_id)
            # Update the MongoDB document with reviews
            collection.update_one({'_id': movie['_id']}, {'$set': {'reviews': reviews}})
        else:
            print(f"No TMDB entry found for {movie_title}")
    else:
        print("Movie title is missing in the record")

movies_cursor.close()

## Check Movies where reviews are empty in the TMDB Dataset

In [ ]:
movies_with_empty_reviews = collection.find({'reviews': {'$exists': True, '$size': 0}})
empty_reviews_count = 0

# Loop through the cursor and process each movie
for movie in movies_with_empty_reviews:
    print(movie.get('movie_title'))  # Print the movie title
    empty_reviews_count += 1

# Print total count of movies with empty reviews
print("Total movies with empty reviews:", empty_reviews_count)

## Add Sentiment to the TMDB Dataset reviews

In [ ]:
classifier = pipeline('sentiment-analysis', model='sarahai/movie-sentiment-analysis')
# Fetch movies
movies = collection.find({}, no_cursor_timeout=True).batch_size(50)

def clean_review(text):
    text = re.sub(r'\w+\d+ \w+ \d+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

try:
    for movie in movies:
        print(movie['movie_title'])
        if 'reviews' in movie:
            for review in movie['reviews']:
                cleaned_review = clean_review(review['content'])  
                result = classifier(cleaned_review[:512]) 
                if result[0]['label'] == 'LABEL_1':
                    sentiment = "positive"
                else:
                    sentiment = "negative"
                review['sentiment'] = sentiment
                collection.update_one({'_id': movie['_id']}, {'$set': {'reviews': movie['reviews']}})
finally:
    movies.close()

print("Sentiments added to all reviews.")

## Check Similar movies in both IMDB user reviews Dataset and TMDB user reviews Dataset

In [ ]:
# Establish a connection to MongoDB
client = MongoClient('mongodb://localhost:27017/')

# Access the databases and collections
movies_db = client['movies']  # Database with 'name'
# rotten_movies = client['rotten_movies']  # Database with 'movie_title'

# Access the movies collections in both databases
movies_collection1 = movies_db['imdb_reviews']  # Collection with 'name'
movies_collection2 = movies_db['tmdb_reviews']  # Collection with 'movie_title'

# Fetch movies from both collections
movies1 = set(doc['name'].strip().lower() for doc in movies_collection1.find({}, {'name': 1}))
movies2 = set(doc['movie_title'].strip().lower() for doc in movies_collection2.find({}, {'movie_title': 1}))

# Find intersections (similar movies)
similar_movies = movies1.intersection(movies2)

# Print similar movies
for movie in similar_movies:
    print(movie)
print(f"Total number of similar movies: {len(similar_movies)}")

## Replace the similar movies records of TMDB data with IMDB data as IMDB data has more reviews

In [ ]:
# Establish a connection to MongoDB
client = MongoClient('mongodb://localhost:27017/')

# Access the databases
movies_db = client['new_data']
# rotten_movies = client['rotten_movies']

# Collections access
movies_collection1 = movies_db['imdb_reviews']
movies_collection2 = movies_db['tmdb_reviews']

# Create a new collection in 'rotten_movies' database
new_collection = movies_db['imdb_tmdb_reviews']

# Step 1: Duplicate movies_collection2 in the new collection, excluding the '_id' field
for document in movies_collection2.find({}, {'_id': 0}):  # Exclude '_id' to allow MongoDB to generate new ones
    new_collection.insert_one(document)

# Prepare for updating by fetching data in a suitable format
# Mapping by normalized movie title from movies_collection1
movies1_docs = {doc['name'].strip().lower(): doc for doc in movies_collection1.find()}

# Step 2: Update similar movies records in the new collection
for doc in new_collection.find():
    normalized_title = doc.get('movie_title', '').strip().lower()
    if normalized_title in movies1_docs:
        replacement_doc = movies1_docs[normalized_title].copy()
        replacement_doc.pop('_id', None)  # Remove '_id' to avoid inserting it
        new_collection.replace_one({'_id': doc['_id']}, replacement_doc)

print("Database update complete.")


In [ ]:
from pymongo import MongoClient
import ast

# Establish a connection to MongoDB
client = MongoClient('mongodb://localhost:27017/')

# Access the database and collection
movies_db = client['movies']
new_collection = movies_db['imdb_tmdb_reviews']

# Find documents with the second format
documents = new_collection.find({'movie_title': {'$exists': True}})

# Process each document
for doc in documents:
    try:
        print(doc['movie_title'])
        # Rename movie_title to name and make other adjustments
        updated_document = {
            'name': doc.pop('movie_title'),
            'genres': ast.literal_eval(doc['genres']),
            'year': int(doc['original_release_date'][:4]),
            'release_date': doc.pop('original_release_date')
        }
        
        # Update the document in the collection
        result = new_collection.update_one({'_id': doc['_id']}, {'$set': updated_document})
        if result.matched_count == 0:
            print(f"No document matches the id: {doc['_id']}")
        elif result.modified_count == 0:
            print(f"Document with id: {doc['_id']} was not updated (it may already be up to date).")
        else:
            print(f"Document with id: {doc['_id']} updated successfully.")
    except Exception as e:
        print(f"Error updating document {doc['_id']}: {str(e)}")

print("Documents have been updated.")


## Drop unnecessary columns

In [ ]:
# Establish a connection to MongoDB
client = MongoClient('mongodb://localhost:27017/')

# Access the database and collection
movies_db = client['movies']
new_collection = movies_db['imdb_tmdb_reviews']

# Define a list of keys to remove from the documents
keys_to_remove = ['movie_title', 'movie_rated', 'run_length', 'rating', 
                  'num_raters', 'num_reviews', 'directors', 'actors', 'production_company', 'original_release_date']

# Find all documents in the collection
documents = new_collection.find()

for doc in documents:
    if "movie_title" in doc:
        print(doc["movie_title"])
    else:
        print("Document does not have a movie_title.")

    # Construct an unset operation
    unset_operation = {key: "" for key in keys_to_remove if key in doc}

    # Update the document to remove the keys
    if unset_operation:
        result = new_collection.update_one({'_id': doc['_id']}, {'$unset': unset_operation})
        if result.matched_count == 0:
            print(f"No document matches the id: {doc['_id']}")
        elif result.modified_count == 0:
            print(f"Document with id: {doc['_id']} was not updated (it may already be up to date or no fields to unset).")
        else:
            print(f"Document with id: {doc['_id']} updated successfully.")
    else:
        print("No relevant keys to remove for this document.")

print("Update process is complete.")

## Fetch actors from TMDB datasets

In [ ]:
# Establish a connection to MongoDB
client = MongoClient('mongodb://localhost:27017/')

# Access the database and collection
movies_db = client['movies']
new_collection = movies_db['imdb_tmdb_reviews']

def get_movie_actors(movie_name, api_key):
    search_url = f"https://api.themoviedb.org/3/search/movie?api_key={api_key}&query={movie_name}"
    try:
        search_response = requests.get(search_url).json()
        if not search_response['results']:
            return "Movie not found"

        movie_id = search_response['results'][0]['id']
        credits_url = f"https://api.themoviedb.org/3/movie/{movie_id}/credits?api_key={api_key}"
        credits_response = requests.get(credits_url).json()

        actors = [cast['name'] for cast in credits_response.get('cast', []) if 'name' in cast]
        return actors
    except requests.RequestException as e:
        print(f"Request failed: {e}")
        return None

api_key = '6748a0069777d79ba648978d81157873'  # Replace with your TMDb API key

try:
    movies = new_collection.find({}, no_cursor_timeout=True).batch_size(50)
    for movie in movies:
        movie_name = movie['name']  # Assuming each movie document has a 'name' field
        print(f"Processing {movie_name}")
        actors = get_movie_actors(movie_name, api_key)
        if actors and actors != "Movie not found":
            new_collection.update_one({'_id': movie['_id']}, {'$set': {'actors': actors}})
finally:
    movies.close()

print("Actors added to all movies.")

## Save data to Database

In [35]:
# # to be run if skipping the above code

# d = pd.read_json('./datasets/intermediate_results/imdb_tmdb_reviws_with_sentiments.json')
# d.drop(columns=['_id'], inplace=True)
# save_to_database('imdb_tmdb_reviews_with_sentiments', d)

## Fetch Data from Databases

In [45]:
client = MongoClient('mongodb://127.0.0.1:27017')
db = client["movies"]
imdb_tmdb_reviews_with_sentiments_collection = db['imdb_tmdb_reviews_with_sentiments']

# Fetch movies
imdb_tmdb_reviews_with_sentiments_cursor = imdb_tmdb_reviews_with_sentiments_collection.find({}, no_cursor_timeout=True).batch_size(50)

# read movies from database into a dataframe
imdb_tmdb_reviews_with_sentiments_df = pd.DataFrame(list(imdb_tmdb_reviews_with_sentiments_cursor))


# close the cursor
imdb_tmdb_reviews_with_sentiments_cursor.close()

imdb_tmdb_reviews_with_sentiments_df.drop(columns=['_id'], inplace=True)



/Users/gaurav/anaconda3/envs/prog_for_ai/lib/python3.9/site-packages/pymongo/collection.py:1697: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout
  return Cursor(self, *args, **kwargs)


## Data Preprocessing

In [46]:
imdb_tmdb_reviews_with_sentiments_df['genres'] = [genre[0].split('; ')[:-1] for genre in imdb_tmdb_reviews_with_sentiments_df['genres']]
# Remove country information from 'release_date' column
imdb_tmdb_reviews_with_sentiments_df['release_date'] = imdb_tmdb_reviews_with_sentiments_df['release_date'].str.split('\(', expand=True)[0].str.strip()

# Try to convert to datetime using the format "11 July 1997"
imdb_tmdb_reviews_with_sentiments_df['release_date'] = pd.to_datetime(imdb_tmdb_reviews_with_sentiments_df['release_date'], format="%d %B %Y", errors='coerce')

# Handle exceptions separately
exceptions = imdb_tmdb_reviews_with_sentiments_df[imdb_tmdb_reviews_with_sentiments_df['release_date'].isna()]['release_date']
for date_str in exceptions:
    parsed_date = pd.to_datetime(date_str, format="%B %Y", errors='coerce')
    if not pd.isnull(parsed_date):
        imdb_tmdb_reviews_with_sentiments_df.loc[imdb_tmdb_reviews_with_sentiments_df['release_date'] == date_str, 'release_date'] = parsed_date

imdb_tmdb_reviews_with_sentiments_df = imdb_tmdb_reviews_with_sentiments_df[imdb_tmdb_reviews_with_sentiments_df['release_date'].dt.year >= 2010].reset_index(drop=True)

# Explode the 'reviews' column to create multiple rows
imdb_tmdb_reviews_with_sentiments_df = imdb_tmdb_reviews_with_sentiments_df.explode('reviews')

# Extract 'content' and 'sentiment' from the exploded 'reviews' column
imdb_tmdb_reviews_with_sentiments_df['user_review'] = imdb_tmdb_reviews_with_sentiments_df['reviews'].apply(lambda x: x['content'])
imdb_tmdb_reviews_with_sentiments_df['user_sentiment'] = imdb_tmdb_reviews_with_sentiments_df['reviews'].apply(lambda x: 1 if x['sentiment'] == 'positive' else 0)

# Drop the original 'reviews' column
imdb_tmdb_reviews_with_sentiments_df.drop(columns=['reviews'], inplace=True)

imdb_tmdb_reviews_with_sentiments_df['release_date'] = pd.to_datetime(imdb_tmdb_reviews_with_sentiments_df['release_date'])
imdb_tmdb_reviews_with_sentiments_df.sort_values(by='release_date', inplace=True)

## Visualizations

In [50]:
# Flatten the 'genres' column
flat_genre_df = imdb_tmdb_reviews_with_sentiments_df.explode('genres')

# Calculate total number of positive and negative reviews for each genre
genre_reviews_counts = flat_genre_df.groupby(['genres', 'user_sentiment'])['name'].count().reset_index(name='num_reviews')

# Pivot the table to have positive and negative reviews as columns
genre_reviews_pivot = genre_reviews_counts.pivot(index='genres', columns='user_sentiment', values='num_reviews').fillna(0)

# Calculate percentage of positive and negative reviews for each genre
genre_reviews_pivot['total_reviews'] = genre_reviews_pivot.sum(axis=1)
genre_reviews_pivot['positive_percentage'] = (genre_reviews_pivot[1] / genre_reviews_pivot['total_reviews']) * 100
genre_reviews_pivot['negative_percentage'] = (genre_reviews_pivot[0] / genre_reviews_pivot['total_reviews']) * 100

# Create stacked bar chart
fig = go.Figure()
fig.add_trace(go.Bar(x=genre_reviews_pivot.index, y=genre_reviews_pivot['positive_percentage'], name='Positive', marker_color='green'))
fig.add_trace(go.Bar(x=genre_reviews_pivot.index, y=genre_reviews_pivot['negative_percentage'], name='Negative', marker_color='red'))

# Update layout
fig.update_layout(barmode='stack', title='Percentage of Positive and Negative Reviews by Genre', xaxis_title='Genre', yaxis_title='Percentage')

# Show plot
fig.show()

In [51]:
import pandas as pd
import plotly.graph_objects as go

# Explode the 'genres' column to create separate rows for each genre
exploded_df = imdb_tmdb_reviews_with_sentiments_df.explode('genres')


# Group data by year and genre, and calculate sentiment distribution
grouped_data = exploded_df.groupby(['year', 'genres', 'user_sentiment']).size().unstack(fill_value=0)

# Plot histograms for each genre over the years
for genre in exploded_df['genres'].unique():
    genre_data = grouped_data.loc[grouped_data.index.get_level_values('genres') == genre]

    fig = go.Figure()
    for sentiment in [0, 1]:  # Negative and positive sentiments
        values = genre_data[sentiment].tolist()
        fig.add_trace(go.Bar(x=genre_data.index.get_level_values('year'), y=values, name='Negative' if sentiment == 0 else 'Positive'))

    fig.update_layout(title=f'Sentiment Distribution for Genre: {genre} Over the Years', barmode='stack')
    fig.update_xaxes(title='Year')
    fig.show()

In [53]:
# Explode the 'genres' column to create separate rows for each genre
exploded_df = imdb_tmdb_reviews_with_sentiments_df.explode('genres')

exploded_df['year'] = exploded_df['release_date'].dt.year

# Group data by year and genre, and calculate sentiment distribution
grouped_data = exploded_df.groupby(['year', 'genres', 'user_sentiment']).size().unstack(fill_value=0)

# Calculate overall, positive, and negative sentiment counts for each genre for each year
genre_sentiment_counts = {}
for genre in exploded_df['genres'].unique():
    genre_sentiment_counts[genre] = {}
    for year in exploded_df['year'].unique():
        try:
            genre_data = grouped_data.loc[(year, genre), :]
            overall_count = genre_data.sum()
            positive_count = genre_data[1]
            negative_count = genre_data[0]
        except KeyError:
            # If the combination doesn't exist, set counts to zero
            overall_count = 0
            positive_count = 0
            negative_count = 0
        genre_sentiment_counts[genre][year] = {'overall': overall_count, 'positive': positive_count, 'negative': negative_count}

# Convert the dictionary to a DataFrame for easier visualization
genre_sentiment_counts_df = pd.DataFrame.from_dict({(i, j): genre_sentiment_counts[i][j] 
                                                     for i in genre_sentiment_counts.keys() 
                                                     for j in genre_sentiment_counts[i].keys()},
                                                   orient='index')


# Stacked Bar Chart
fig = go.Figure()
for genre in genre_sentiment_counts_df.index.levels[0]:
    genre_data = genre_sentiment_counts_df.loc[genre]
    fig.add_trace(go.Bar(x=genre_data.index, y=genre_data['positive'], name=f'{genre} - Positive'))
    fig.add_trace(go.Bar(x=genre_data.index, y=genre_data['negative'], name=f'{genre} - Negative'))

fig.update_layout(barmode='stack', title='Overall Sentiment Counts for Each Genre Over the Years',
                  xaxis_title='Year', yaxis_title='Sentiment Count')
fig.show()

# Line Plot
fig = go.Figure()
for genre in genre_sentiment_counts_df.index.levels[0]:
    genre_data = genre_sentiment_counts_df.loc[genre]
    fig.add_trace(go.Scatter(x=genre_data.index, y=genre_data['positive'], mode='lines', name=f'{genre} - Positive'))
    fig.add_trace(go.Scatter(x=genre_data.index, y=genre_data['negative'], mode='lines', name=f'{genre} - Negative'))

fig.update_layout(title='Trend of Sentiment Counts for Each Genre Over the Years',
                  xaxis_title='Year', yaxis_title='Sentiment Count')
fig.show()


# Area Chart
fig = go.Figure()
for genre in genre_sentiment_counts_df.index.levels[0]:
    genre_data = genre_sentiment_counts_df.loc[genre]
    fig.add_trace(go.Scatter(x=genre_data.index, y=genre_data['positive'], fill='tozeroy', mode='none',
                             name=f'{genre} - Positive'))
    fig.add_trace(go.Scatter(x=genre_data.index, y=genre_data['negative'], fill='tozeroy', mode='none',
                             name=f'{genre} - Negative'))

fig.update_layout(title='Trend of Sentiment Counts for Each Genre Over the Years',
                  xaxis_title='Year', yaxis_title='Sentiment Count', hovermode='x')
fig.show()

# Grouped Bar Chart
fig = go.Figure()

for genre in genre_sentiment_counts_df.index.levels[0]:
    genre_data = genre_sentiment_counts_df.loc[genre]
    fig.add_trace(go.Bar(x=genre_data.index, y=genre_data['positive'], 
                         name=f'{genre} - Positive', offset=-0.25))
    fig.add_trace(go.Bar(x=genre_data.index, y=genre_data['negative'], 
                         name=f'{genre} - Negative', offset=0.25))

fig.update_layout(barmode='group', title='Sentiment Counts for Each Genre Within Each Year',
                  xaxis_title='Year', yaxis_title='Sentiment Count')
fig.show()


# IMDB Movies

In [2]:
imdb_movies = pd.read_csv('./datasets/movies_metadata/TMDB_all_movies.csv')

In [3]:
imdb_movies.shape

(927261, 25)

## Saving to Database

In [10]:
imdb_movies = imdb_movies.drop(
    columns=['id', 'revenue', 'runtime', 'budget', 'imdb_id', 'original_language', 'overview', 'popularity', 'tagline', 
    'production_countries', 'spoken_languages', 'director_of_photography', 'writers', 'producers',
       'music_composer', 'vote_average', 'vote_count', 'status', 'original_title', 'production_companies', 'director']
)
imdb_movies.to_csv('./datasets/intermediate_results/imdb_movies.csv', index=False)
# saving to database
save_to_database('imdb_movies', imdb_movies)

## Reading data from Database

In [46]:
client = MongoClient('mongodb://127.0.0.1:27017')
db = client["movies"]
imdb_movies_collection = db['imdb_movies']

# Fetch movies
imdb_movies_cursor = imdb_movies_collection.find({}, no_cursor_timeout=True).batch_size(50)

# read movies from database into a dataframe
imdb_movies_df = pd.DataFrame(list(imdb_movies_cursor))

# close the cursor
imdb_movies_cursor.close()

imdb_movies_df.drop(columns=['_id'], inplace=True)

/Users/gaurav/anaconda3/envs/prog_for_ai/lib/python3.9/site-packages/pymongo/collection.py:1697: UserWarning:

use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout



## Data Preprocessing

In [11]:
def imdb_movies_data_preprocessing(input_df):
    """
    Preprocess the imdb_movies dataset
    1. Drop columns that are not needed
    2. Drop rows with missing values
    3. Drop duplicates
    4. Convert release_date to datetime
    5. Sort by release_date
    6. Reset index
    7. Return the preprocessed dataframe
    """
    df = input_df.copy()
    df = df.dropna()
    df = df.drop_duplicates(subset=['title'])
    df['release_date'] = pd.to_datetime(df['release_date'])
    df = df.sort_values(by='release_date')
    df = df.reset_index(drop=True)
    df['genres'] = df['genres'].apply(lambda x: x.split(', ') if ',' in x else [x])
    df = df[df['release_date'].dt.year >= 2010].reset_index(drop=True)
    return df

In [12]:
imdb_movies_df = imdb_movies_data_preprocessing(imdb_movies_df)

In [13]:
imdb_movies_df

,title,release_date,genres,cast
0,Vigil 5.4,2010-01-01,[Documentary],Rebecca Belmore
1,Ruhling 4 Ever,2010-01-01,[Documentary],"Pit Trenz, Markus Rühl, Simone Rühl"
2,Lychee Thieves,2010-01-01,"[Comedy, Drama]","Pomaika'i Brown, Linda Sato, Wayne Pyle, Scott..."
3,When All the Leaves Are Gone,2010-01-01,[Drama],"Marie-Josée Corneau, Daphnée Vincent, Justine ..."
4,After-Life,2010-01-01,[Drama],"Christopher Garcia, Matthew Dearing, Annie Boo..."
...,...,...,...,...
201001,Avatar 4,2029-12-19,"[Science Fiction, Adventure, Fantasy]","Cliff Curtis, Sam Worthington, Jemaine Clement..."
201002,La Persistencia de la Paranoia,2030-05-16,"[Comedy, History]","Adriano Leone, Abraham Cerpa, Iván Cibreiro, N..."
201003,Portal Quest 3: TDB,2030-07-01,"[Action, Adventure, Animation, Fantasy]",AvatarKage
201004,Avatar 5,2031-12-17,"[Adventure, Science Fiction, Fantasy]","Stephen Lang, Zoe Saldaña, Joel David Moore, C..."


In [54]:
# Filter the dataset for a specific actor/actress
actor_name = "Robert Downey Jr."
actor_data = imdb_movies_df[imdb_movies_df['cast'].str.contains(actor_name, na=False)]

# Group the data by release year
grouped_data = actor_data.groupby(actor_data['release_date'].dt.year)['genres'].apply(lambda x: x.explode())

# Loop through each year
for year, group in grouped_data.groupby(level=0):
    # Count the occurrences of each genre for the current year
    genre_counts = group.value_counts(normalize=True) * 100
    
    # Create a pie chart for the current year
    fig = go.Figure(data=go.Pie(
        labels=genre_counts.index,
        values=genre_counts.values,
        marker_colors=px.colors.qualitative.Pastel
    ))
    
    # Customize layout
    fig.update_layout(
        title=f'Percentage of Genres Worked in by {actor_name} in {year}',
        title_font_size=20,
        title_font_family="Arial",
        title_font_color="navy",
        font=dict(
            family="Arial",
            size=14,
            color="darkslategray"
        )
    )
    
    # Show the plot
    fig.show()

In [57]:
# Explode the 'genres' column to separate rows for each genre
exploded_df = imdb_movies_df.explode('genres')

# Group by release month and genre, considering unique movie titles
exploded_df['release_month'] = exploded_df['release_date'].dt.month
seasonal_genre_counts_unique = exploded_df.groupby(['release_month', 'genres'])['title'].nunique().unstack(fill_value=0)

# Reset index to ensure 'release_month' is a column and not an index
seasonal_genre_counts_unique = seasonal_genre_counts_unique.reset_index()

# Convert release_month to categorical for proper ordering on the x-axis
seasonal_genre_counts_unique['release_month'] = pd.Categorical(seasonal_genre_counts_unique['release_month'], categories=range(1, 13), ordered=True)

# Melt the dataframe for Plotly
melted_df = pd.melt(seasonal_genre_counts_unique, id_vars=['release_month'], var_name='genre', value_name='num_movies')

# Create the seasonal trends plot using Plotly
fig = px.bar(melted_df, x='release_month', y='num_movies', color='genre', barmode='stack',
             category_orders={'release_month': list(range(1, 13))},
             labels={'release_month': 'Month', 'num_movies': 'Number of Movies', 'genre': 'Genre'},
             title='Seasonal Trends by Genre by months',
             color_discrete_sequence=px.colors.qualitative.Pastel)
fig.update_layout(xaxis={'tickmode': 'array', 'tickvals': list(range(1, 13)),
                         'ticktext': ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                                      'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']})
fig.show()

In [59]:
# Explode the 'genres' column to separate rows for each genre
exploded_df = imdb_movies_df.explode('genres')

# Extract year from release date
exploded_df['release_year'] = exploded_df['release_date'].dt.year

# Group by release year and genre, considering unique movie titles
seasonal_genre_counts_unique = exploded_df.groupby(['release_year', 'genres'])['title'].nunique().unstack(fill_value=0)

# Reset index to ensure 'release_year' is a column and not an index
seasonal_genre_counts_unique = seasonal_genre_counts_unique.reset_index()

# Melt the dataframe for Plotly
melted_df = pd.melt(seasonal_genre_counts_unique, id_vars=['release_year'], var_name='genre', value_name='num_movies')

# Create the seasonal trends plot using Plotly
fig = px.bar(melted_df, x='release_year', y='num_movies', color='genre', barmode='stack',
             labels={'release_year': 'Year', 'num_movies': 'Number of Movies', 'genre': 'Genre'},
             title='Seasonal Trends by Genre (Unique Movie Titles)',
             color_discrete_sequence=px.colors.qualitative.Pastel)
fig.show()

In [60]:
# get count of positive review and negative reviews for each movie bname from user_reviews in a dictionary like {'movie_title': {'positive': 10, 'negative': 5}}
user_reviews_sentiment_counts = imdb_tmdb_reviews_with_sentiments_df.groupby(['name', 'user_sentiment'])['user_review'].count().unstack(fill_value=0).reset_index()
user_reviews_sentiment_counts.columns.name = None
user_reviews_sentiment_counts.rename(columns={0: 'negative', 1: 'positive'}, inplace=True)
user_reviews_sentiment_counts = user_reviews_sentiment_counts.set_index('name').to_dict(orient='index')

In [76]:
# get count of positive review and negative reviews for each movie bname from user_reviews in a dictionary like {'movie_title': {'positive': 10, 'negative': 5}}
critics_reviews_sentiment_counts = rotten_tomatoes_critic_reviews_df.groupby(['movie_title', 'critic_sentiment'])['critic_review'].count().unstack(fill_value=0).reset_index()
critics_reviews_sentiment_counts.columns.name = None
critics_reviews_sentiment_counts.rename(columns={0: 'negative', 1: 'positive'}, inplace=True)
critics_reviews_sentiment_counts = critics_reviews_sentiment_counts.set_index('movie_title').to_dict(orient='index')

In [77]:
# for each movie title add user_reviews_sentiment_counts and critics_reviews_sentiment_counts
imdb_movies_df['user_reviews_sentiment_counts'] = imdb_movies_df['title'].map(user_reviews_sentiment_counts)
imdb_movies_df['critics_reviews_sentiment_counts'] = imdb_movies_df['title'].map(critics_reviews_sentiment_counts)

In [78]:
# drop columns where user_reviews_sentiment_counts and critics_reviews_sentiment_counts are both NAN
imdb_movies_df = imdb_movies_df.dropna(subset=['user_reviews_sentiment_counts', 'critics_reviews_sentiment_counts'])

In [126]:
imdb_movies_df.head(2)

,title,release_date,genres,cast,user_reviews_sentiment_counts,critics_reviews_sentiment_counts,release_year,sentiment_ratio,user_sentiment_ratio,critics_sentiment_ratio,user_sentiment_ratio_positive,user_sentiment_ratio_negative,critics_sentiment_ratio_positive,critics_sentiment_ratio_negative
1070,Black Panther,2010-01-18,[Animation],"Reginald Hudlin, Kerry Washington, Rick D. Was...","{'negative': 9, 'positive': 16}","{'negative': 113, 'positive': 399}",2010,0.772812,0.64,0.779297,0.64,0.36,0.779297,0.220703
1356,Get Out,2010-02-01,"[Comedy, Animation]","Philippe Peythieu, Jacques Briere","{'negative': 4, 'positive': 21}","{'negative': 187, 'positive': 199}",2010,0.535280,0.84,0.515544,0.84,0.16,0.515544,0.484456


## Save to Database

In [79]:
save_to_database('imdb_movies_with_sentiments', imdb_movies_df)

In [85]:
imdb_movies_df['release_year'] = pd.to_datetime(imdb_movies_df['release_date']).dt.year

# Explode the genres column
movies_exploded = imdb_movies_df.explode('genres')

# Group by genre, year, and sentiment type
user_sentiment = movies_exploded.groupby(['genres', 'release_year'])['user_reviews_sentiment_counts'].apply(lambda x: {'positive': sum(d.get('positive', 0) for d in x), 'negative': sum(d.get('negative', 0) for d in x)}).reset_index()
critic_sentiment = movies_exploded.groupby(['genres', 'release_year'])['critics_reviews_sentiment_counts'].apply(lambda x: {'positive': sum(d.get('positive', 0) for d in x), 'negative': sum(d.get('negative', 0) for d in x)}).reset_index()

# Plotting
genres = movies_exploded['genres'].unique()
# Plotting
genres = user_sentiment['genres'].unique()
for genre in genres:
    df_user_genre = user_sentiment[user_sentiment['genres'] == genre]
    df_critic_genre = critic_sentiment[critic_sentiment['genres'] == genre]
    
    user_positive_trace = go.Bar(
        x=df_user_genre[df_user_genre['level_2'] == 'positive']['release_year'],
        y=df_user_genre[df_user_genre['level_2'] == 'positive']['user_reviews_sentiment_counts'],
        name='User Positive'
    )
    user_negative_trace = go.Bar(
        x=df_user_genre[df_user_genre['level_2'] == 'negative']['release_year'],
        y=df_user_genre[df_user_genre['level_2'] == 'negative']['user_reviews_sentiment_counts'],
        name='User Negative'
    )
    critic_positive_trace = go.Bar(
        x=df_critic_genre[df_critic_genre['level_2'] == 'positive']['release_year'],
        y=df_critic_genre[df_critic_genre['level_2'] == 'positive']['critics_reviews_sentiment_counts'],
        name='Critic Positive'
    )
    critic_negative_trace = go.Bar(
        x=df_critic_genre[df_critic_genre['level_2'] == 'negative']['release_year'],
        y=df_critic_genre[df_critic_genre['level_2'] == 'negative']['critics_reviews_sentiment_counts'],
        name='Critic Negative'
    )
    
    layout = go.Layout(
        title=f'Sentiments Over Years for {genre} Genre',
        xaxis=dict(title='Release Year'),
        yaxis=dict(title='Sentiment Counts'),
        barmode='group',
        showlegend=True,
        legend=dict(x=0, y=1)
    )
    
    fig = go.Figure(data=[user_positive_trace, user_negative_trace, critic_positive_trace, critic_negative_trace], layout=layout)
    fig.show()

/var/folders/h2/8cgfpxjx39ggvhldlqk085t40000gn/T/ipykernel_50240/1774388385.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [125]:
# Step 1: Calculate the total number of movie releases for each year
total_releases_per_year = imdb_movies_df.groupby('release_year').size()

# Step 2: Calculate the total number of movie releases for each genre from 2010 to 2015 and 2015 to 2020
movie_releases_per_genre = imdb_movies_df.explode('genres').groupby(['genres', 'release_year']).size().unstack(fill_value=0)

# Step 3: Calculate the sentiment ratio for each genre for user reviews
def calculate_user_sentiment_ratio(row):
    positive_reviews = row['user_reviews_sentiment_counts']['positive']
    negative_reviews = row['user_reviews_sentiment_counts']['negative']
    total_reviews = positive_reviews + negative_reviews
    if total_reviews == 0:
        return 0, 0
    return positive_reviews / total_reviews, negative_reviews / total_reviews

# Apply user sentiment ratio calculation to each row
imdb_movies_df['user_sentiment_ratio_positive'], imdb_movies_df['user_sentiment_ratio_negative'] = zip(*imdb_movies_df.apply(calculate_user_sentiment_ratio, axis=1))

# Step 4: Calculate the sentiment ratio for each genre for critics reviews
def calculate_critics_sentiment_ratio(row):
    positive_reviews = row['critics_reviews_sentiment_counts']['positive']
    negative_reviews = row['critics_reviews_sentiment_counts']['negative']
    total_reviews = positive_reviews + negative_reviews
    if total_reviews == 0:
        return 0, 0
    return positive_reviews / total_reviews, negative_reviews / total_reviews

# Apply critics sentiment ratio calculation to each row
imdb_movies_df['critics_sentiment_ratio_positive'], imdb_movies_df['critics_sentiment_ratio_negative'] = zip(*imdb_movies_df.apply(calculate_critics_sentiment_ratio, axis=1))

# Step 5: Calculate the percentage of movie releases for each genre
movie_releases_per_genre['percentage_2010_2015'] = movie_releases_per_genre[2010] / total_releases_per_year.loc[2010:2015].sum() * 100
movie_releases_per_genre['percentage_2015_2020'] = movie_releases_per_genre[2015] / total_releases_per_year.loc[2015:2020].sum() * 100

# Step 6: Check if there's an increase in the percentage of movie releases for each genre from 2015 to 2020 compared to 2010 to 2015
movie_releases_per_genre['increase'] = movie_releases_per_genre['percentage_2015_2020'] > movie_releases_per_genre['percentage_2010_2015']

# Step 7: Create the plot
fig = go.Figure()

fig.add_trace(go.Bar(
    x=movie_releases_per_genre.index,
    y=movie_releases_per_genre['percentage_2010_2015'],
    name='2010-2015 Movie Releases',
    marker_color='skyblue'
))

fig.add_trace(go.Bar(
    x=movie_releases_per_genre.index,
    y=movie_releases_per_genre['percentage_2015_2020'],
    name='2015-2020 Movie Releases',
    marker_color='lightgreen'
))

# Add scatter plots for user sentiment ratio and critics sentiment ratio (positive and negative separately)
fig.add_trace(go.Scatter(
    x=movie_releases_per_genre.index,
    y=imdb_movies_df['user_sentiment_ratio_positive'],
    name='User Sentiment Ratio (Positive)',
    mode='lines+markers',
    line=dict(color='blue', width=2)
))

fig.add_trace(go.Scatter(
    x=movie_releases_per_genre.index,
    y=imdb_movies_df['user_sentiment_ratio_negative'],
    name='User Sentiment Ratio (Negative)',
    mode='lines+markers',
    line=dict(color='cyan', width=2)
))

fig.add_trace(go.Scatter(
    x=movie_releases_per_genre.index,
    y=imdb_movies_df['critics_sentiment_ratio_positive'],
    name='Critics Sentiment Ratio (Positive)',
    mode='lines+markers',
    line=dict(color='red', width=2)
))

fig.add_trace(go.Scatter(
    x=movie_releases_per_genre.index,
    y=imdb_movies_df['critics_sentiment_ratio_negative'],
    name='Critics Sentiment Ratio (Negative)',
    mode='lines+markers',
    line=dict(color='orange', width=2)
))

# Step 8: Customize layout
fig.update_layout(
    title='Percentage of Movie Releases and Sentiment Ratios by Genre (2010-2015 vs 2015-2020)',
    xaxis_title='Genre',
    yaxis_title='Percentage / Sentiment Ratio',
    barmode='group'
)

/var/folders/h2/8cgfpxjx39ggvhldlqk085t40000gn/T/ipykernel_50240/3802324618.py:25: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/h2/8cgfpxjx39ggvhldlqk085t40000gn/T/ipykernel_50240/3802324618.py:37: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [132]:
# Step 6: Create the narrative description
narrative_descriptions = []
for genre, row in movie_releases_per_genre.iterrows():
    if row['increase']:
        narrative_descriptions.append(f"The number of releases for the genre '{genre}' increased from 2015 to 2020 compared to the period from 2010 to 2015 due to positive reviews.")
    else:
        narrative_descriptions.append(f"The number of releases for the genre '{genre}' decreased from 2015 to 2020 compared to the period from 2010 to 2015 due to negative reviews.")

# Step 7: Display the narrative descriptions
for description in narrative_descriptions:
    print(description)

The number of releases for the genre 'Action' increased from 2015 to 2020 compared to the period from 2010 to 2015 due to positive reviews.
The number of releases for the genre 'Adventure' increased from 2015 to 2020 compared to the period from 2010 to 2015 due to positive reviews.
The number of releases for the genre 'Animation' decreased from 2015 to 2020 compared to the period from 2010 to 2015 due to negative reviews.
The number of releases for the genre 'Comedy' increased from 2015 to 2020 compared to the period from 2010 to 2015 due to positive reviews.
The number of releases for the genre 'Crime' increased from 2015 to 2020 compared to the period from 2010 to 2015 due to positive reviews.
The number of releases for the genre 'Documentary' decreased from 2015 to 2020 compared to the period from 2010 to 2015 due to negative reviews.
The number of releases for the genre 'Drama' increased from 2015 to 2020 compared to the period from 2010 to 2015 due to positive reviews.
The number 